# Noteboook for testing view classification

In [2]:
import sys
import os
import pandas as pd

projectdir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(projectdir, 'src'))


In [4]:
import random
import sys

import os
import subprocess
import time
import cv2
import pydicom
from optparse import OptionParser
from shutil import rmtree

import tensorflow as tf
import numpy as np
from scipy.misc import imread

from d00_utils.dcm_utils import dcmdir_to_jpgs_for_classification


In [5]:
import pandas as pd
import os
from json import load
from sqlalchemy import create_engine
from sqlalchemy.schema import CreateSchema
from sqlalchemy import inspect
import tempfile
import gc

query = "SELECT COLUMN_NAME, DATA_TYPE \
            FROM INFORMATION_SCHEMA.COLUMNS \
            WHERE \
            TABLE_NAME = 'meta_lite';""


SyntaxError: invalid syntax (<ipython-input-5-b1d1d5a8ed7e>, line 10)